In [ ]:
import numpy as np
import adsorbate_thermo
import matplotlib.pyplot as plt
import rmgpy.species
import copy
import cantera as ct

# Get NASA fit using rmg_tools, Katrin's code, and directly from Cantera

In [ ]:
# Get nasa fit using rmg_tools, Katrin's code, and directly from Cantera
molecular_weight = 18.02
frequencies = [49.5, 68.6, 73.6, 102.0, 437.6, 452.9, 1596.3, 3675.6, 3787.0]
composition = {'H': 2, 'O': 1, 'C': 0, 'N': 0}
heat_of_formation_0K = -259.05  # kJ/mol

my_calc = adsorbate_thermo.AdsorbateThermoCalc(molecular_weight, frequencies, composition, heat_of_formation_0K, twoD_gas=True)
# Get NASA from rmg_tools
nasa_rmg_tools = my_calc.get_thermo()

# Get NASA using Katrin's code
a_low_katrin, a_high_katrin = my_calc.get_thermo2()


# get NASA from Cantera
# https://www.mit.edu/course/2/2.62/cantera/doc/html/NasaPoly2_8h_source.html#l00081
gas1 = ct.Solution('ptcombust.cti')
surf1 = ct.Interface('ptcombust.cti', 'Pt_surf', [gas1])
print(surf1.species()[2])
midpoint = surf1.species()[2].thermo.coeffs[0]
a_high_ct = surf1.species()[2].thermo.coeffs[1:8]
a_low_ct = surf1.species()[2].thermo.coeffs[8:]

In [ ]:
# Get thermo for the NASA polynomials using Katrin's NASA2thermo
cp_fit_rmg_tools, h_fit_rmg_tools, s_fit_rmg_tools = my_calc.get_thermo_from_NASA2(nasa_rmg_tools.polynomials[0].coeffs, nasa_rmg_tools.polynomials[1].coeffs)
cp_fit_katrin, h_fit_katrin, s_fit_katrin = my_calc.get_thermo_from_NASA2(a_low_katrin, a_high_katrin)
cp_fit_ct_katrin, h_fit_ct_katrin, s_fit_ct_katrin = my_calc.get_thermo_from_NASA2(a_low_ct, a_high_ct)

In [ ]:
# convert other NASA polynomials to RMG NASA objects
nasa_katrin = copy.deepcopy(nasa_rmg_tools)
nasa_katrin.polynomials[0].coeffs = a_low_katrin
nasa_katrin.polynomials[1].coeffs = a_high_katrin

nasa_ct = copy.deepcopy(nasa_rmg_tools)
nasa_ct.polynomials[0].coeffs = a_low_ct
nasa_ct.polynomials[1].coeffs = a_high_ct


In [ ]:
# Get thermo from cantera
gas1 = ct.Solution('ptcombust.cti')
surf1 = ct.Interface('ptcombust.cti', 'Pt_surf', [gas1])
gas1.TPX = gas1.TPX = 300, 101325, {'H2O': 1.0}

Cp_ct = np.zeros(len(my_calc.temperatures))
H_ct = np.zeros(len(my_calc.temperatures))
S_ct = np.zeros(len(my_calc.temperatures))

Cp_rmg = np.zeros(len(my_calc.temperatures))
H_rmg = np.zeros(len(my_calc.temperatures))
S_rmg = np.zeros(len(my_calc.temperatures))

Cp_katrin_rmg = np.zeros(len(my_calc.temperatures))
H_katrin_rmg = np.zeros(len(my_calc.temperatures))
S_katrin_rmg = np.zeros(len(my_calc.temperatures))

Cp_ct_rmg = np.zeros(len(my_calc.temperatures))
H_ct_rmg = np.zeros(len(my_calc.temperatures))
S_ct_rmg = np.zeros(len(my_calc.temperatures))

for i, T in enumerate(my_calc.temperatures):
    surf1.TPX = T, 101325, {'H2O(S)': 1.0}
    Cp_ct[i] =  surf1.cp_mole / 1000.0
    H_ct[i] = surf1.enthalpy_mole / 1000.0  # units are in J/mol and we want kJ/mol
    S_ct[i] = surf1.entropy_mole / 1000.0
    
    # Get thermo for rmg_tools NASA polynomials using RMG
    Cp_rmg[i] = nasa_rmg_tools.get_heat_capacity(T) / 1000.0
    H_rmg[i] = nasa_rmg_tools.get_enthalpy(T) / 1000.0  # units are in J/mol and we want kJ/mol
    S_rmg[i] = nasa_rmg_tools.get_entropy(T) / 1000.0
    
    # Get thermo for katrin NASA polynomials using RMG
    Cp_katrin_rmg[i] = nasa_katrin.get_heat_capacity(T)
    H_katrin_rmg[i] = nasa_katrin.get_enthalpy(T)
    S_katrin_rmg[i] = nasa_katrin.get_entropy(T)
    
    # Get thermo for cantera NASA polynomials using RMG
    Cp_ct_rmg[i] = nasa_ct.get_heat_capacity(T)
    H_ct_rmg[i] = nasa_ct.get_enthalpy(T)
    S_ct_rmg[i] = nasa_ct.get_entropy(T)


# Plot the resulting thermo: Cp, H, S

## Heat Capacity Cp

In [ ]:
# Plot Cp
fig = plt.figure()
ax = plt.subplot(111)

# Katrin's NASA2thermo
plt.plot(my_calc.temperatures, cp_fit_katrin, linestyle=(0,(5,10)), label='Katrin NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, cp_fit_rmg_tools / 1000.0, linestyle=(10,(5,10)), label='RMG NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, cp_fit_ct_katrin, linestyle=(3,(5,10)), label='Cantera NASA fit and Katrin NASA2thermo')

# rmg tools NASA2thermo
plt.plot(my_calc.temperatures, Cp_rmg, linestyle=(7,(5,10)), label='RMG NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, Cp_katrin_rmg, linestyle=(7,(5,10)), label='Katrin NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, Cp_ct_rmg, linestyle=(7,(5,10)), label='Cantera NASA fit and RMG NASA2thermo')

plt.plot(my_calc.temperatures, Cp_ct, linestyle=(10,(10,10)), label='Cantera')
plt.title('Heat Capacity')
plt.xlabel('T [k]')
plt.ylabel('Cp [J/(mol K)]')
ax.legend(loc='right', bbox_to_anchor=(1.8, 0.5))

In [ ]:
# We can group them as follows: the left uses Cantera's thermo, and the right uses Katrin's thermo
# there are some discontinuities when you switch the fitting and NASA2thermo methods,
# but in general, the RMG NASA2thermo is the same as Katrin's NASA2thermo for heat capacity


# Plot Cp
fig = plt.figure(figsize=(10, 4), dpi=80)
ax = plt.subplot(121)

# Cantera thermo
plt.plot(my_calc.temperatures, cp_fit_ct_katrin, linestyle=(3,(5,10)), label='Cantera NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, Cp_ct_rmg, linestyle=(7,(5,10)), label='Cantera NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, Cp_ct, linestyle=(10,(10,10)), label='Cantera')
plt.title('Heat Capacity')
plt.xlabel('T [k]')
plt.ylabel('Cp [J/(mol K)]')
ax.legend()

# rmg tools thermo
ax = plt.subplot(122)
plt.plot(my_calc.temperatures, cp_fit_katrin, linestyle=(0,(5,10)), label='Katrin NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, cp_fit_rmg_tools / 1000.0, linestyle=(10,(5,10)), label='RMG NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, Cp_rmg, linestyle=(7,(5,10)), label='RMG NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, Cp_katrin_rmg, linestyle=(7,(5,10)), label='Katrin NASA fit and RMG NASA2thermo')

plt.title('Heat Capacity')
plt.xlabel('T [k]')
plt.ylabel('Cp [J/(mol K)]')
ax.legend()

## Enthalpy H

In [ ]:
# Plot H
fig = plt.figure()
ax = plt.subplot(111)

# Katrin's NASA2thermo
plt.plot(my_calc.temperatures, h_fit_katrin, linestyle=(0,(5,10)), label='Katrin NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, h_fit_rmg_tools / 1000.0, linestyle=(10,(5,10)), label='RMG NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, h_fit_ct_katrin, linestyle=(3,(5,10)), label='Cantera NASA fit and Katrin NASA2thermo')

# rmg tools NASA2thermo
plt.plot(my_calc.temperatures, H_rmg, linestyle=(7,(5,10)), label='RMG NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, H_katrin_rmg, linestyle=(7,(5,10)), label='Katrin NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, H_ct_rmg, linestyle=(7,(5,10)), label='Cantera NASA fit and RMG NASA2thermo')

plt.plot(my_calc.temperatures, H_ct, linestyle=(10,(10,10)), label='Cantera')
plt.title('Enthalpy')
plt.xlabel('T [k]')
plt.ylabel('H [J/mol]')
ax.legend(loc='right', bbox_to_anchor=(1.8, 0.5))


In [ ]:
# Once again we can divide it into Cantera and Katrin thermo and see that the NASA2thermo agrees

fig = plt.figure(figsize=(10, 4), dpi=80)
ax = plt.subplot(121)

# Cantera thermo
plt.plot(my_calc.temperatures, h_fit_ct_katrin, linestyle=(3,(5,10)), label='Cantera NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, H_ct_rmg, linestyle=(7,(5,10)), label='Cantera NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, H_ct, linestyle=(10,(10,10)), label='Cantera')
plt.title('Enthalpy')
plt.xlabel('T [k]')
plt.ylabel('H [J/mol]')
ax.legend()

# rmg tools thermo
ax = plt.subplot(122)
plt.plot(my_calc.temperatures, h_fit_katrin, linestyle=(0,(5,10)), label='Katrin NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, h_fit_rmg_tools / 1000.0, linestyle=(10,(5,10)), label='RMG NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, H_rmg, linestyle=(7,(5,10)), label='RMG NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, H_katrin_rmg, linestyle=(7,(5,10)), label='Katrin NASA fit and RMG NASA2thermo')

plt.title('Enthalpy')
plt.xlabel('T [k]')
plt.ylabel('H [J/mol]')
ax.legend()

## Entropy S

In [ ]:
# Plot S

fig = plt.figure()
ax = plt.subplot(111)

# Katrin's NASA2thermo
plt.plot(my_calc.temperatures, s_fit_katrin, linestyle=(0,(5,10)), label='Katrin NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, s_fit_rmg_tools / 1000.0, linestyle=(10,(5,10)), label='RMG NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, s_fit_ct_katrin, linestyle=(3,(5,10)), label='Cantera NASA fit and Katrin NASA2thermo')

# rmg tools NASA2thermo
plt.plot(my_calc.temperatures, S_rmg, linestyle=(7,(5,10)), label='RMG NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, S_katrin_rmg, linestyle=(7,(5,10)), label='Katrin NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, S_ct_rmg, linestyle=(7,(5,10)), label='Cantera NASA fit and RMG NASA2thermo')

plt.plot(my_calc.temperatures, S_ct, linestyle=(10,(10,10)), label='Cantera')
plt.title('Entropy')
plt.xlabel('T [k]')
plt.ylabel('S [J/(mol K)]')
ax.legend(loc='right', bbox_to_anchor=(1.8, 0.5))


In [ ]:
# Divide it into Cantera and Katrin thermo

fig = plt.figure(figsize=(10, 4), dpi=80)
ax = plt.subplot(121)

# Cantera thermo
plt.plot(my_calc.temperatures, s_fit_ct_katrin, linestyle=(3,(5,10)), label='Cantera NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, S_ct_rmg, linestyle=(7,(5,10)), label='Cantera NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, S_ct, linestyle=(10,(10,10)), label='Cantera')
plt.title('Entropy')
plt.xlabel('T [k]')
plt.ylabel('S [J/(mol K)]')
ax.legend()

# rmg tools thermo
ax = plt.subplot(122)
plt.plot(my_calc.temperatures, s_fit_katrin, linestyle=(0,(5,10)), label='Katrin NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, s_fit_rmg_tools / 1000.0, linestyle=(10,(5,10)), label='RMG NASA fit and Katrin NASA2thermo')
plt.plot(my_calc.temperatures, S_rmg, linestyle=(7,(5,10)), label='RMG NASA fit and RMG NASA2thermo')
plt.plot(my_calc.temperatures, S_katrin_rmg, linestyle=(12,(5,10)), label='Katrin NASA fit and RMG NASA2thermo')

plt.title('Entropy')
plt.xlabel('T [k]')
plt.ylabel('S [J/(mol K)]')
ax.legend()